In [24]:
import numpy as np
import pandas as pd
import random
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as k
import os

In [25]:
def load(paths, verbose=-1):
    data = list()
    labels = list()
    # Loop over input images
    for (i,imgpath) in enumerate(paths):
        #  load image and extract label
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        #  scale image
        data.append(image/255)
        labels.append(label)
        #  show update
        if verbose > 0 and i > 0 and (i+1) % verbose ==  0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
        #  return tuple mof data and label
    return data,labels

In [26]:
#  declare path
# img_path = '/home/nextg3/Documents/FederatedLearning/FL_final/MNIST_training'
img_path = '/home/nextg3/Documents/Thesis/Code/FL/MNIST_training'
# Get path list
image_paths = list(paths.list_images(img_path))
#  apply function
image_list, label_list = load(image_paths, verbose = 10000)
#  Binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)
# split train test
X_train,X_test, y_train, y_test = train_test_split(image_list,
                                                   label_list,
                                                   test_size=0.1,
                                                   random_state=37)

[INFO] processed 10000/60000
[INFO] processed 20000/60000
[INFO] processed 30000/60000
[INFO] processed 40000/60000
[INFO] processed 50000/60000
[INFO] processed 60000/60000


In [27]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    '''returns a dictionary with keys as client names and values as data shards - (image, label)
        args:
    
                '''
    #  create list of client names
    client_names = ['{}_{}'.format(initial,i) for i in range(num_clients)]
    #  randomize data
    data = list(zip(image_list,label_list))
    random.shuffle(data)
    #  place data at each client
    size = len(data)//num_clients
    shards = [data[i:i+size] for i in range(0, size*num_clients, size)]

    # no of shrads should equa no of clients
    assert(len(shards)==len(client_names))

    return {client_names[i]:shards[i] for i in range(len(client_names))}

In [28]:
#  create clients
clients = create_clients(X_train,y_train,num_clients=10, initial='client')

In [29]:
def batch_data(data_shard, bs=32):
    '''client data shard -> tfds object
        args:
            shard
            batch size
        return
            tfds object'''
    #  sep data and label
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data),list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [30]:
# Process and batch data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#  Processed and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(len(y_test))

In [31]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [32]:
lr = 0.01
coms_round = 100
loss = 'categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate = lr,
                momentum = 0.9
                )

In [33]:
def weight_scaling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    # get bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    # print('bs',bs)
    # Calculater the total data points across the clients
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # print('glocal_count: ',global_count)
    # get total no of data points held by the client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    # print('local_count: ',local_count)
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''Function for scaling a model's weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar*weight[i])
    return weight_final


In [34]:
def  sum_scaled_weights(scaled_weight_list):
    '''Returns the sum of the listed scaled weights.
    scaled average of the weights'''
    avg_grad = list()
    #  get the average grad over all client gradients
    for grad_list_tupel in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tupel, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


def test_model(X_test, Y_test, model, coms_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    # logits=model.predict(X_test, batch_size=100)
    logits=model.predict(X_test)
    loss = cce(y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis  = 1), tf.argmax(Y_test, axis = 1))
    print('coom_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [35]:
# Initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784,10)

comms_round = 10

# start global training loop
for comm_round in range(comms_round):
    #  get the global model's weights - initial weights for all local models
    global_weights = global_model.get_weights()

    # List to collect local model weights after scaling
    scaled_local_weight_list = list()

    # Randomize client data using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # Loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784,10)
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)
        # set weights of the global model as weights of the local modelk
        local_model.set_weights(global_weights)

        # Fit local model with client data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to the list
        # scaling_factor = weight_scaling_factor(clients_batched, client)
        # scaled_weights = scale_model_weights(local_model.get_weights(),scaling_factor)
        # scaled_local_weightsmlp_local = SimpleMLP()
        # local_model = smlp_local.build(784,10)
        # local_model.compile(loss=loss,
        #                     optimizer=optimizer,
        #                     metrics=metrics)
        # # set weights of the global model as weights of the local modelk
        # local_model.set_weights(global_weights)

        # # Fit local model with client data
        # local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to the list
        scaling_factor = weight_scaling_factor(clients_batched, client)
        # save loca_weights
        local_model.save_weights('{}_weights.h5'.format(client))
        scaled_weights = scale_model_weights(local_model.get_weights(),scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # Clear session to clean memory
        k.clear_session()
    
    # Get the average overall model
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(average_weights)

    # Test globalmodel
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


188/188 [==============================] - 0s 637us/step
coom_round: 0 | global_acc: 90.433% | global_loss: 1.6292821168899536
188/188 [==============================] - 0s 778us/step
coom_round: 1 | global_acc: 92.450% | global_loss: 1.5880060195922852
188/188 [==============================] - 0s 610us/step
coom_round: 2 | global_acc: 93.283% | global_loss: 1.5702006816864014
188/188 [==============================] - 0s 611us/step
coom_round: 3 | global_acc: 94.500% | global_loss: 1.5560517311096191
188/188 [==============================] - 0s 692us/step
coom_round: 4 | global_acc: 94.783% | global_loss: 1.5468307733535767
188/188 [==============================] - 0s 647us/step
coom_round: 5 | global_acc: 95.183% | global_loss: 1.5397067070007324
188/188 [==============================] - 0s 608us/step
coom_round: 6 | global_acc: 95.633% | global_loss: 1.5326088666915894
188/188 [==============================] - 0s 634us/step
coom_round: 7 | global_acc: 95.717% | global_loss: 1.5

In [36]:
str(average_weights)

'[<tf.Tensor: shape=(784, 200), dtype=float32, numpy=\narray([[-0.05319114, -0.019837  ,  0.06939308, ..., -0.04295979,\n        -0.06557755,  0.04630484],\n       [-0.05632387,  0.07539415,  0.03932199, ..., -0.04766873,\n         0.02875637,  0.06874508],\n       [-0.03600975,  0.01135702, -0.07762122, ..., -0.01793904,\n        -0.04088428,  0.06563687],\n       ...,\n       [-0.01409819, -0.01114139,  0.00948918, ..., -0.06594594,\n         0.05415611, -0.05522961],\n       [-0.00039982, -0.05005729, -0.04146854, ...,  0.07435285,\n        -0.07084067,  0.03079193],\n       [-0.06574259,  0.04187585, -0.04515645, ..., -0.06178195,\n         0.02637964,  0.03797305]], dtype=float32)>, <tf.Tensor: shape=(200,), dtype=float32, numpy=\narray([ 0.00357905, -0.00328432,  0.06080082,  0.01316852,  0.01048604,\n       -0.00469753, -0.00777614,  0.03704596,  0.01874357,  0.02811878,\n        0.0321062 ,  0.01702331,  0.02388108,  0.00559376,  0.05674131,\n        0.01446631,  0.03809216,  0

### Export architecture to json file

In [37]:

json_string = global_model.to_json()

In [38]:
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "batch_input_shape": [null, 784], "units": 200, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Activation", "config": {"name": "activation", "trainable": true, "dtype": "float32", "activation": "relu"}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 200, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "Gl

### Export weights as h5 file

In [39]:
global_model.save_weights('weights.h5')

In [41]:
import h5py
f = h5py.File('weights.h5', 'r')
# print(list(f.keys()))
# # will get a list of layer names which you can use as index
# d = f['dense']['dense_1']['kernel:0']
# # <HDF5 dataset "kernel:0": shape (128, 1), type "<f4">
# d.shape == (128, 1)
# d[0] == array([-0.14390108], dtype=float32)

In [42]:
f

<HDF5 file "weights.h5" (mode r)>

### Export weights as a list

In [40]:
import pickle

with open('weights.pkl', 'wb') as f:
    pickle.dump(average_weights, f)

Try Global model aggregation

In [ ]:
average_weights = sum_scaled_weights(scaled_local_weight_list)
# update global model
global_model.set_weights(average_weights)